In [1]:
from webscrapper import WebScraper
import fitz
import re
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ['OPENAI_API_KEY'] = "sk-proj-hOBF9U5T1HTBMEMrUexUT3BlbkFJsg7dgTVSB5X0Sg8hUm2H"

#### Code to extract answers from pdf

In [3]:
def extract_qa_from_pdf(pdf_path):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    
    # Extract text from all pages
    text = ""
    for page in doc:
        text += page.get_text()
    
    # Close the document
    doc.close()
    
    lines = text.split('\n')
    qa_dict = {}
    current_speaker = None
    current_message = []
    first_user_statement = True
    current_question = None

    for line in lines:
        if line.strip() == 'Assistant:':
            if current_speaker == 'User' and not first_user_statement:
                qa_dict[current_question] = ' '.join(current_message).strip()
            current_speaker = 'Assistant'
            current_message = []
        elif line.strip() == 'User:':
            if current_speaker == 'Assistant':
                current_question = ' '.join(current_message).strip()
            current_speaker = 'User'
            current_message = []
            if first_user_statement:
                first_user_statement = False
        elif line.strip():
            current_message.append(line.strip())

    # Handle the last message
    if current_speaker == 'User' and not first_user_statement and current_question:
        qa_dict[current_question] = ' '.join(current_message).strip()
    qa_dict.pop(None,None)
    return qa_dict

In [ ]:
# Usage
pdf_path = 'conversation.pdf'
qa_dictionary = extract_qa_from_pdf(pdf_path)
qa_dictionary

In [5]:
for question, answer in qa_dictionary.items():
    print(f"Q: {question}")
    print(f"A: {answer}")
    print()

Q: Hello! Thank you for taking the time to speak with me today about the Entry-Level Machine Learning Engineer position at G-Research. To start, could you tell me about your experience with machine learning, particularly any projects or coursework you've completed in this area?
A: I'm sure I have a total of six months of experience with machine learning after I entered the University of Kent. Right now I'm mainly working on Numpy and Pandas. Numpy is to analyze image information and Pandas to extract data from CSVs.

Q: Thank you for sharing that information. Could you elaborate on a specific machine learning project you've worked on, perhaps one that involved Numpy or Pandas? What was the goal of the project, and what challenges did you face?
A: I'm sure so one of the projects that I worked on was classifying the minst data set of images. Yeah basically it was done. I extracted the training set from the minst data, used a CNN algorithm to find out features from the images and then use

#### Code to Decompose asnwers to find relevant sites on the internet

In [6]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-queries related to an answer by an interview candidate. \n
You will be provided the original question for context. The goal is to find the accuracy of the answer. For this
you will break down the answer into a set of sub-problems / sub-queries that can be used as a search string in google to find the relevant information. \n
Here is the original question: {question}
Generate multiple search queries related to: {answer} \n
Output (2 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [8]:
# LLM
llm = ChatOpenAI(temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))
content = ""
# Run
for question, answer in qa_dictionary.items():
    queries = generate_queries_decomposition.invoke({"question":question,"answer":answer})
    print(queries)
    for query in queries:
        scraper = WebScraper(answer,2)
        # Call the method to get the file paths of the scraped data
        content = "".join([content,scraper.get_scraped_data()])
content

['1. "University of Kent machine learning projects coursework"', '2. "Numpy image analysis Pandas CSV data extraction"']
['1. "Classifying MNIST dataset images using CNN algorithm project details"', '2. "Challenges faced in implementing CNN algorithm for image classification project"']
['1. "Fine-tuning LLM model for chatbot applications"', '2. "Applications of LLM models in chatbots"']


'\n\n--- Content from https://madewithml.com/courses/foundations/pandas/ ---\n\nSkip to content\n\n\n\n\n\n\n⚡️ Checkout our new End-to-end LLM Workflows Guide\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMade With ML\n\n\n\n\nPandas for Machine Learning\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nInitializing search\n\n\n\n\n\n\n\n\n\n\n\n\n\nGokuMohandas/MadeWithML\n\n\n\n\n\n\n\n\n\n\n\nHome\n\n\n\n\nAbout\n\n\n\n\nCourse\n\n\n\n\nFoundations\n\n\n\n\nSubscribe\n\n\n\n\nCommunity\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMade With ML\n\n\n\n\n\n\n\nGokuMohandas/MadeWithML\n\n\n\n\n\n\nHome\n\n\n\n\nAbout\n\n\n\n\n\nCourse\n\n\n\n\n\nCourse\n\n\n\n\nLessons\n\n\n\n\n\n🎨 \xa0 Design\n\n\n\n\n\n🎨 \xa0 Design\n\n\n\n\nSetup\n\n\n\n\nProduct\n\n\n\n\nSystems\n\n\n\n\n\n\n\n\n🔢 \xa0 Data\n\n\n\n\n\n🔢 \xa0 Data\n\n\n\n\nPreparation\n\n\n\n\nExploration\n\n\n\n\nPreprocessing\n\n\n\n\nDistributed\n\n\n\n\n\n\n\n\n📈 \xa0 Model\n\n\n\n\n\n📈 \xa0 Model\n\n\n\n\nTraining\n\n\n\n\nTracking\n\n\n\n\nTuning\n\n

: 

#### Storing the relevant documents in a vector store

In [ ]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(content)

# Index

vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

#### Checking accuracy with  background context.

In [ ]:
# Prompt
template = """Here is the answer you need to check:

\n --- \n {answer} \n --- \n

Here is any available background question + answer + accuracy percentage + feedback:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context, your own knowledge and background question + answer + accuracy percentage + feedback
on the subject matter to get the accuracy of the answer. A percentage accuracy score and also note down the 
places that the answer was inaccurate and give feedback for those places
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [ ]:

def format_qa_pair(question, answer, feedback):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\nFeedback: {feedback}\n\n"
    return formatted_string.strip()

# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

q_a_pairs = ""
for question, answer in qa_dictionary.items():
    
    rag_chain = (
    {"context": itemgetter("answer") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    feedback = rag_chain.invoke({"question":question,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(question,answer,feedback)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [ ]:
q_a_pairs